# Hyperparameter Optimization

For this exercise, we will have a look at Hyperparameter Optimization --
instead of just choosing the best type of machine learning model, we also want
to choose the best hyperparameter setting for a task. The end result (i.e. the
predictive performance) is again not important; how you get there is.

Your deliverable will be a report, written in a style that it
would be suitable for inclusion in an academic paper as the "Experimental
Setup" section or similar. If unsure, check an academic paper of your choice,
for example [this one](https://www.eecs.uwyo.edu/~larsko/papers/pulatov_opening_2022-1.pdf). The
level of detail should be higher than in a typical academic paper though. Your
report should be at most five pages, including references and figures but
excluding appendices. It should have the following structure:
- Introduction: What problem are you solving, how are you going to solve it.
- Dataset Description: Describe the data you're using, e.g. how many features and observations, what are you predicting, any missing values, etc.
- Experimental Setup: What specifically are you doing to solve the problem, i.e.\ what programming languages and libraries, how are you processing the data, what machine learning algorithms are you considering and what hyperparameters and value ranges, what measures you are using to evaluate them, what hyperparameter optimization method you chose, etc.
- Results: Description of what you observed, including plots. Compare
  performance before and after tuning, and show the best configuration.
- Code: Add the code you've used as a separate file.

Your report must contain enough detail to reproduce what you did without the
code. If in doubt, include more detail.

There is no required format for the report. You could, for example, use an
iPython notebook.

## Data and Setup

We will have a look at the [Wine Quality
dataset](https://archive-beta.ics.uci.edu/dataset/186/wine+quality). Choose the
one that corresponds to your preference in wine. You may also use a dataset of
your choice, for example one that's relevant to your research.

Choose a small number of different machine learning algorithms and
hyperparameters, along with value ranges, for each. You can use implementations
of AutoML systems (e.g. auto-sklearn), scientific papers, or the documentation
of the library you are using to determine the hyperparameters to tune and the
value ranges. Note that there is not only a single way to do this, but define a
reasonable space (e.g. don't include whether to turn on debug output, or random
forests with 1,000,000 trees, or tune the loss function). Your hyperparameter
search space should be so large that you cannot simply run a grid search.

Determine the best machine learning algorithm and hyperparameter setting for
your dataset. Make sure to optimize both the type of machine learning algorithm
and the hyperparameters at the same time (do not first choose the best ML
algorithm and then optimize its hyperparameters). Choose a suitable
hyperparameter optimizer; you could also use several and e.g. compare the
results achieved by random search and Bayesian optimization. Make sure that the
way you evaluate model performance avoids bias and overfitting. You could use
statistical tests to make this determination.

## Submission

Add your report and code to this repository. Bonus points if you can set up a
Github action to automatically run the code and generate the report!

## Useful Resources :
- "*Basics of HPO - Example and Practical Hints*" -From the AutoML Course Videos
- https://hpbandster-sklearn.readthedocs.io/en/latest/hpbandster_sklearn.html
- https://www.youtube.com/watch?v=Gol_qOgRqfA
- https://www.youtube.com/watch?v=0wUF_Ov8b0A&t=1058s

## Importing the Dataset as a Pandas Dataframe

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
red_wine_df = pd.read_csv('winequality-red.csv', delimiter=';')

In [ ]:
red_wine_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [ ]:
X = red_wine_df.iloc[:, :-1]
y = red_wine_df['quality']

X.shape, y.shape

((1599, 11), (1599,))

## Importing our Models (SVM Classifier)

### Support Vector Classifier (SVM Classifier)

In [ ]:
from sklearn.svm import SVC

svc_model = SVC()

In [ ]:
svc_model.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

## Hyperparameter Optimization

Methods Used :
- BOHB (Bayesian Optimization with Hyper Band)
- Bayesian Optimization

### BOHB (Bayesian Optimization with Hyper Band)
- Used to quicken hyperparameter optimization of the SVM Classifier model.

In [ ]:
# Comment out this line to download required package for the HPO method:
!pip install hpbandster-sklearn

In [ ]:
from hpbandster_sklearn import HpBandSterSearchCV
import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH

# Construct the hyperparameter distribution:

hyperparameter_distribution = {
    "C" : CS.Float("C", bounds=(0.1, 100.0), default=1.0),
    "kernel" : CS.Categorical("kernel", ["linear", "poly", "rbf", "sigmoid"], default="rbf"),
    "degree" : CS.Integer("degree", bounds=(1, 20), default=3),
    "gamma" : CS.Categorical("gamma", ["scale", "auto"], default="scale"),
    "coef0" : CS.Float("coef0", bounds=(0.0, 1.0), default=0.0),
    "tol" : CS.Float("tol", bounds=(0.001, 1.0), default=0.001),
}

RANDOM_STATE = 90

param_distributions = CS.ConfigurationSpace(
    seed=RANDOM_STATE,
    name='SVC_hpo_space',
    space = hyperparameter_distribution
)

bohb_search = HpBandSterSearchCV(
    svc_model,
    param_distributions,
    scoring='accuracy',
    cv=10,
    optimizer='bohb',
    random_state=RANDOM_STATE,
    n_jobs=1,
    n_iter=50,
    verbose=1,
)

In [ ]:
# Due to some speed issues and after googling the issue...
# Some resources suggest scaling the data, so...
# Carry out important pre-processing for SVC :

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

In [ ]:
bohb_search.fit(X_scaled, y)

WORKER: start listening for jobs
INFO:hpbandster_sklearn.HpBandSterSearchCV:WORKER: start listening for jobs
HBMASTER: adjusted queue size to (0, 1)
INFO:hpbandster_sklearn.HpBandSterSearchCV:HBMASTER: adjusted queue size to (0, 1)
HBMASTER: starting run at 1711266820.784447
INFO:hpbandster_sklearn.HpBandSterSearchCV:HBMASTER: starting run at 1711266820.784447
WORKER: start processing job (0, 0, 0)
INFO:hpbandster_sklearn.HpBandSterSearchCV:WORKER: start processing job (0, 0, 0)
WORKER: registered result for job (0, 0, 0) with dispatcher
INFO:hpbandster_sklearn.HpBandSterSearchCV:WORKER: registered result for job (0, 0, 0) with dispatcher
WORKER: start processing job (0, 0, 1)
INFO:hpbandster_sklearn.HpBandSterSearchCV:WORKER: start processing job (0, 0, 1)
WORKER: registered result for job (0, 0, 1) with dispatcher
INFO:hpbandster_sklearn.HpBandSterSearchCV:WORKER: registered result for job (0, 0, 1) with dispatcher
WORKER: start processing job (0, 0, 2)
INFO:hpbandster_sklearn.HpBand

HpBandSterSearchCV(cv=10, estimator=SVC(), n_iter=50, n_jobs=1,
                   param_distributions=Configuration space object:
  Hyperparameters:
SVC_hpo_space
    C, Type: UniformFloat, Range: [0.1, 100.0], Default: 1.0
    coef0, Type: UniformFloat, Range: [0.0, 1.0], Default: 0.0
    degree, Type: UniformInteger, Range: [1, 20], Default: 3
    gamma, Type: Categorical, Choices: {scale, auto}, Default: scale
    kernel, Type: Categorical, Choices: {linear, poly, rbf, sigmoid}, Default: rbf
    tol, Type: UniformFloat, Range: [0.001, 1.0], Default: 0.001
,
                   random_state=90, scoring='accuracy', verbose=1)

In [ ]:
bohb_search.best_score_ , bohb_search.best_params_

(0.596627358490566,
 {'C': 0.9806390302563345,
  'coef0': 0.9484348047607323,
  'degree': 19,
  'gamma': 'scale',
  'kernel': 'rbf',
  'tol': 0.31713221942837955})

### Bayesian Optimization

In [ ]:
# Comment out this line to install the necessary library for Bayesian Optimization:
!pip install baytune

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.8 MB/s eta 0:00:00


In [ ]:
models = {
    'SVC': SVC
}

In [ ]:
from sklearn.model_selection import cross_val_score

def scoring_function(model_name, hyperparameter_values):
    model_class = models[model_name]
    model_instance = model_class(**hyperparameter_values)
    scores = cross_val_score(
        cv=10,
        estimator=model_instance,
        X=X_scaled,
        y=y,
        scoring='accuracy',
    )

    return scores.mean()

In [ ]:
from baytune.tuning import Tunable
from baytune.tuning import hyperparams as hp

tunables = {
    'SVC': Tunable({
        'C': hp.FloatHyperParam(min=0.1, max=100, default=1.0),
        'kernel' : hp.CategoricalHyperParam(["linear", "poly", "rbf", "sigmoid"], default="rbf"),
        'degree' : hp.IntHyperParam(min=1, max=20, default=3),
        'gamma' : hp.CategoricalHyperParam(["scale", "auto"], default="scale"),
        "coef0" : hp.FloatHyperParam(min=0.0, max=1.0, default=0.0),
        "tol" : hp.FloatHyperParam(min=0.001, max=1.0, default=0.001),
    }),
}

In [ ]:
from baytune import BTBSession

session = BTBSession(
    tunables=tunables,
    scorer=scoring_function,
    verbose=True,
)

In [ ]:
best_result = session.run(200)

best_result

  0%|          | 0/200 [00:00<?, ?it/s]

Exception in thread discover_workers:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/Pyro4/core.py", line 511, in connect_and_handshake
    sock = socketutil.createSocket(connect=connect_location,
  File "/usr/local/lib/python3.10/dist-packages/Pyro4/socketutil.py", line 307, in createSocket
    sock.connect(connect)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/Pyro4/core.py", line 2009, in _locateNS
    proxy._pyroBind()
  File "/usr/local/lib/python3.10/dist-packages/Pyro4/core.py", line 408, in _pyroBind
    return self.__pyroCreateConnection(True)
  File "/usr/local/lib/python3.10/dist-packages/Pyro4/core.py", line 596, in __pyroCreateConnection
    connect_and_handshake(conn)
  File "/usr/local/lib/python3.10/dist-packages/Pyro4/core.py", line 549, in connect_and_handshake
    raise c

{'id': 'dcb05f11c1f4852dc94407670a19723a',
 'name': 'SVC',
 'config': {'C': 0.6827484819888097,
  'kernel': 'rbf',
  'degree': 13,
  'gamma': 'scale',
  'coef0': 0.6124410983213924,
  'tol': 0.4541981726388633},
 'score': 0.5972484276729559}